# 図の作成例

Sample notebook to show waveforms from CSV file.  
It is NOT recommended to move this notebook into other directories.

|Date   |Commit history|
|:------|:-------------|
|04/28 2021|First created by Yasunori Sawaki (Kyoto Univ.)|
|05/16     |Interactive plot using [`Bokeh`](https://bokeh.org/)|

To run each cell, only to enter `Shift+Enter` (cf. `Ctrl+Enter`)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 必要なパッケージのインポート

Pythonでは，パッケージを`import`することで使用可能になる

In [ ]:
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta
import argparse
import requests

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd


notebookname = requests.get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
print('notebookname ->', notebookname)

### パス設定

1. 左側のタブ`ファイル`から`EarthScienceExperiment`という名前のフォルダを探す
2. フォルダ上で右クリックして「パスをコピー」を選択
3. 下の`projectBaseDir`にパスを貼り付ける（セルは自動的に更新される）

Look for the directory named `EarthScienceExperiment` and right click for **`Copy path`**.

In [ ]:
#@title Paste directory path / ↓パスを貼り付ける↓ {run: "auto"}

projectBaseDir = '/content/drive/MyDrive/ItoLab/EarthScienceExperiment' #@param {type:"string"}
projectBaseDir = Path(projectBaseDir)

#@markdown 例：projectBaseDir : `"/content/drive/MyDrive/EarthScienceExperiment"`

if not (projectBaseDir/notebookname).exists():
    raise FileNotFoundError(f'Path not found. Check again projectBaseDir: {projectBaseDir}')

os.chdir(projectBaseDir)

### 描画パッケージ`matplotlib`のおまじない

In [ ]:
# @markdown `plt.rcParams`の設定　<u>このセルも実行</u>してください

plt.rc('figure', figsize=[8,6], facecolor='w', dpi=100, )

## savefig
plt.rc(
    'savefig', format='png',
    dpi=plt.rcParams['figure.dpi'], 
    edgecolor=plt.rcParams['figure.edgecolor'],
    facecolor=plt.rcParams['figure.facecolor'],
    bbox='tight', transparent=False,
)
plt.rc('font', family='sans-serif', size='12')

plt.rc('axes', grid=True, linewidth=1.0, axisbelow=True)
plt.rc('axes.grid', axis='both')

plt.rc('lines', linestyle='-', linewidth=0.4, marker=None)

plt.rc('grid', linewidth=0.5, linestyle='--', alpha=0.8)

plt.rc('xtick', direction='in', bottom=True, top=True, labelsize=12)
plt.rc('xtick.major', width=1.0, size=5)
plt.rc('xtick.minor', visible=True, size=2.5)
plt.rc('ytick', direction='in', left=True, right=True, labelsize=12)
plt.rc('ytick.major', width=1.0, size=5)
plt.rc('ytick.minor', visible=True, size=2.5)

plt.rc(
    'legend', markerscale=1, 
    frameon=True, fancybox=False, framealpha=1, #edgecolor='k'
)

#@markdown - 作図で日本語を使う場合は`use_japanise`をチェックして実行
use_japanise = False #@param {type:"boolean"}

if use_japanise:
    !pip install japanize-matplotlib -q
    import japanize_matplotlib
else:
    if "japanize_matplotlib" in sys.modules:
        print(
            '日本語設定を止める場合は，「ランタイムを再起動」し，'
            '「必要なパッケージのインポート」からやり直してください'
        )

## CSVファイルの読み込み

In [ ]:
# @markdown CSVファイルのパスを記述 {run: "auto"}
filepath = "./output/20210420_all.csv" #@param {type:"string"}
filepath = Path(filepath)

# Extract `starttime`
with filepath.open('r') as f:
    f.readline()
    starttime = datetime.strptime(
        f.readline().split()[-1], 
        '%Y-%m-%dT%H:%M:%S.%fZ'
    )

print('starttime: ', starttime)

df = pd.read_csv(
    filepath, 
    comment='#', 
    names=('Time','Ch1','Ch2','Ch3','Ch4')
)

df

- `taxis`: 時間軸の`ndarray`
- `data`: 各チャンネルのデータ`ndarray`

In [ ]:
taxis, *wavdata = df.to_numpy().T

## プロット例

In [ ]:
plt.plot(taxis, wavdata[2]);

In [ ]:
# @title 図の作成例

fig, ax = plt.subplots()

# @markdown - `offset`を設定
offset = 5*10**4 #@param

for ich, data_i in enumerate(wavdata):
    ax.plot(
        taxis, data_i+offset*(ich-1), 
        alpha=0.5, label=f'Ch{ich+1}', zorder=3-0.1*ich,
    )

# @markdown - `xlim`と`ylim`を設定
xlim = (taxis[0], taxis[-1]) #@param
ylim = (-10*10**4, 20*10**4) #@param

ax.set(
    xlim=xlim, ylim=ylim, 
    xlabel=('Time [s]','時間 [秒]')[use_japanise],
    ylabel=('Amplitude [nm/s]', '振幅 [nm/秒]')[use_japanise]
)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(style="sci",  axis="y",scilimits=(0,0))

## Legend
fig.legend()

In [ ]:
import bokeh.plotting as blt
import bokeh.models as bmd
from bokeh.layouts import layout

In [ ]:
# @title Interactive plot {run:'auto'}

# @markdown - `offset`を設定
offset = 50000 #@param

fig = blt.figure(
    title="Interactive plot", 
    x_axis_label='Time [s]', y_axis_label='Amplitude [nm/s]',
    x_range=(taxis[0],taxis[-1]), y_range=(-10**5,2*10**5), 
    plot_width=800, plot_height=500, toolbar_location="above",
)

for ich, data_i in enumerate(wavdata):
    fig.line(
        taxis[::10], data_i[::10]+offset*(ich-1), 
        legend_label=f"Ch{ich+1}", line_width=1., line_alpha=0.3,
        line_color=('blue','orange','green','red')[ich]
    )

xslider = bmd.RangeSlider(
    title="Adjust x-axis range",
    start=taxis[0], end=taxis[-1], step=10,
    value=(fig.x_range.start, fig.x_range.end),  # initial values for slider
)
xslider.js_link("value", fig.x_range, "start", attr_selector=0)
xslider.js_link("value", fig.x_range, "end", attr_selector=1)

# create layout
lay = layout([
    [xslider],
    [fig],
])

# show result
# blt.output_file(filepath.name.replace('csv','html'), title=filepath.name.strip('.csv'))
blt.output_notebook()
blt.show(lay)